# SQLQuery class

Pass a sql string to the SQLQuery class and it will be translated to a Redis query.

**Schema Dictionary:**

In [1]:
schema = {
    "index": {
        "name": "user_simple",
        "prefix": "user_simple_docs",
    },
    "fields": [
        {"name": "user", "type": "tag"},
        {"name": "credit_score", "type": "tag"},
        {"name": "job", "type": "text"},
        {"name": "age", "type": "numeric"},
        {
            "name": "user_embedding",
            "type": "vector",
            "attrs": {
                "dims": 3,
                "distance_metric": "cosine",
                "algorithm": "flat",
                "datatype": "float32"
            }
        }
    ]
}

## Sample Dataset Preparation

Below, create a mock dataset with `user`, `job`, `age`, `credit_score`, and
`user_embedding` fields. The `user_embedding` vectors are synthetic examples
for demonstration purposes.

For more information on creating real-world embeddings, refer to this
[article](https://mlops.community/vector-similarity-search-from-basics-to-production/).

In [2]:
import numpy as np


data = [
    {
        'user': 'john',
        'age': 1,
        'job': 'engineer',
        'credit_score': 'high',
        'user_embedding': np.array([0.1, 0.1, 0.5], dtype=np.float32).tobytes()
    },
    {
        'user': 'mary',
        'age': 2,
        'job': 'doctor',
        'credit_score': 'low',
        'user_embedding': np.array([0.1, 0.1, 0.5], dtype=np.float32).tobytes()
    },
    {
        'user': 'joe',
        'age': 3,
        'job': 'dentist',
        'credit_score': 'medium',
        'user_embedding': np.array([0.9, 0.9, 0.1], dtype=np.float32).tobytes()
    }
]

## Create a `SearchIndex`

With the schema and sample dataset ready, create a `SearchIndex`.

### Bring your own Redis connection instance

This is ideal in scenarios where you have custom settings on the connection instance or if your application will share a connection pool:

In [3]:
from redisvl.index import SearchIndex
from redis import Redis

client = Redis.from_url("redis://localhost:6379")
index = SearchIndex.from_dict(schema, redis_client=client, validate_on_load=True)

### Let the index manage the connection instance

This is ideal for simple cases:

In [4]:
index = SearchIndex.from_dict(schema, redis_url="redis://localhost:6379", validate_on_load=True)

# If you don't specify a client or Redis URL, the index will attempt to
# connect to Redis at the default address "redis://localhost:6379".

### Create the index

Now that we are connected to Redis, we need to run the create command.

In [5]:
index.create(overwrite=True)

>Note that at this point, the index has no entries. Data loading follows.

## Load Data to `SearchIndex`

Load the sample dataset to Redis.

### Validate data entries on load
RedisVL uses pydantic validation under the hood to ensure loaded data is valid and confirms to your schema. This setting is optional and can be configured in the `SearchIndex` class.

In [6]:
keys = index.load(data)

print(keys)

['user_simple_docs:01KFXYVY5NDYGV245XDGFFHW59', 'user_simple_docs:01KFXYVY5QST3G9HQ8F39X39Z7', 'user_simple_docs:01KFXYVY5RF0DJM4JRCXDPF2A2']


## Create a `SQLQuery` Object

In [7]:
from redisvl.query import SQLQuery
from jupyterutils import result_print

sql_query = SQLQuery(
    """
    SELECT user, credit_score, job, age
    FROM user_simple
    WHERE age > 1
    """
)

### Executing the query

In [8]:
results = index.query(sql_query)
result_print(results)

ImportError: sql-redis is required for SQL query support. Install it with: pip install redisvl[sql]

## Cleanup

Below we will clean up after our work. First, you can flush all data from Redis associated with the index by
using the `.clear()` method. This will leave the secondary index in place for future insertions or updates.

But if you want to clean up everything, including the index, just use `.delete()`
which will by default remove the index AND the underlying data.

In [ ]:
# Clear all data from Redis associated with the index
await index.clear()

10

In [ ]:
# Butm the index is still in place
await index.exists()

True

In [ ]:
# Remove / delete the index in its entirety
await index.delete()